In [181]:
import json, folium
import numpy as np
import pandas as pd
import re 

In [182]:
df = pd.read_csv('data/서울시 주요 공원현황.csv', encoding='euc-kr')
df = df[['지역','면적']]
df.columns = ['지역','면적']
df.dropna(subset=['지역','면적'], inplace=True)
df.drop([6], inplace=True)
df.면적 = df.면적.str.replace('총 ','')
df.면적 = df.면적.apply(lambda x: re.split('[㎡m㎥]', x)[0])
df.면적 = df.면적.astype(float).astype(int)
df.isna().sum().sum()

0

In [183]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 127 entries, 0 to 130
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      127 non-null    object
 1   면적      127 non-null    int32 
dtypes: int32(1), object(1)
memory usage: 2.5+ KB


In [184]:
df.head()

,지역,면적
0,중구,2896887
1,강동구,80683
2,과천시,9132690
3,성동구,480994
4,마포구,2284085


In [185]:
df = df.groupby('지역')[['면적']].sum()
df.head()

,면적
지역,
강남구,3626398
강동구,1526387
강북구,3714902
강서구,2194152
과천시,9132690


In [186]:
file = 'data/seoul_geo_simple.json'
geo_data = json.load(open(file, encoding='utf-8'))

In [187]:
map = folium.Map([37.55, 126.98], zoom_start=10, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=df.면적,                    
    columns=[df.index, df.면적],    
    fill_color='PuRd',                  
    key_on='feature.id'                 
).add_to(map)
map

In [188]:
pop = pd.read_csv('../02.Pandas/data/pop.csv', index_col='구별')
pop = pop[['인구수']]
pop.head()

,인구수
구별,
종로구,153789
중구,131787
용산구,237285
성동구,292672
광진구,352627


In [189]:
park = pd.merge(df, pop, left_index=True, right_index=True)
park.head()

,면적,인구수
강남구,3626398,537800
강동구,1526387,466472
강북구,3714902,302563
강서구,2194152,579768
관악구,10431453,499449


In [190]:
park['인당면적'] = (park.면적 / park.인구수).round(2)
park.head()

,면적,인구수,인당면적
강남구,3626398,537800,6.74
강동구,1526387,466472,3.27
강북구,3714902,302563,12.28
강서구,2194152,579768,3.78
관악구,10431453,499449,20.89


In [191]:
map = folium.Map([37.55, 126.98], zoom_start=10, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=park.인당면적,                    
    columns=[park.index, park.인당면적],    
    fill_color='PuRd',                  
    key_on='feature.id'                 
).add_to(map)
map

In [192]:
df1 = pd.read_csv('data/경기도도시공원정보표준데이터.csv',  encoding='euc-kr')
df1 = df1[['소재지지번주소','공원면적']]
df1.columns = ['지역','면적']
df1.dropna(subset=['지역','면적'], inplace=True)
df1.head()

,지역,면적
0,서울특별시 용산구 효창동 287-6,1569.9
23,서울특별시 용산구 이촌동 408,1766.0
24,서울특별시 용산구 한강로1가 30-3,2090.0
25,서울특별시 용산구 신창동 100,257.0
26,서울특별시 용산구 용산동5가 24-2,11879.0


In [193]:
df1['구분'] = df1.지역.apply(lambda x: (x.split()[0]))
df1.head(10)

,지역,면적,구분
0,서울특별시 용산구 효창동 287-6,1569.9,서울특별시
23,서울특별시 용산구 이촌동 408,1766.0,서울특별시
24,서울특별시 용산구 한강로1가 30-3,2090.0,서울특별시
25,서울특별시 용산구 신창동 100,257.0,서울특별시
26,서울특별시 용산구 용산동5가 24-2,11879.0,서울특별시
27,서울특별시 용산구 한강로3가 63-388,2225.0,서울특별시
28,서울특별시 용산구 문배동 24-1,842.0,서울특별시
29,서울특별시 용산구 서계동 276-7,183.0,서울특별시
30,서울특별시 용산구 후암동 406-80,182.0,서울특별시
31,서울특별시 용산구 이촌동 417,889.0,서울특별시


In [194]:
df1.drop(df1[df1['구분'] != "경기도"].index, inplace=True)
df1.head()

,지역,면적,구분
212,경기도 이천시 중리동 15,2526.0,경기도
213,경기도 이천시 중리동 210,3830.0,경기도
214,경기도 이천시 창전동 408,1757.0,경기도
215,경기도 이천시 창전동 414-13,1650.0,경기도
216,경기도 이천시 창전동 423,1650.0,경기도


In [195]:
city = ['수원시','고양시','용인시','성남시','안산시','안양시']
df1['시군구'] = df1.지역.apply(lambda x: ''.join(x.split()[1:3]) if x.split()[1] in city else x.split()[1])
df1.head(100)

,지역,면적,구분,시군구
212,경기도 이천시 중리동 15,2526.0,경기도,이천시
213,경기도 이천시 중리동 210,3830.0,경기도,이천시
214,경기도 이천시 창전동 408,1757.0,경기도,이천시
215,경기도 이천시 창전동 414-13,1650.0,경기도,이천시
216,경기도 이천시 창전동 423,1650.0,경기도,이천시
...,...,...,...,...
477,경기도 용인시 처인구 백암면 근창리 101-8,807.0,경기도,용인시처인구
478,경기도 용인시 처인구 역북동 385-5,391.0,경기도,용인시처인구
479,경기도 용인시 처인구 역북동 386-3,406.0,경기도,용인시처인구
480,경기도 용인시 처인구 포곡읍 삼계리 671-25,1875.0,경기도,용인시처인구


In [196]:
df1 = df1.groupby('시군구')[['면적']].sum()
df1.head()

,면적
시군구,
가평군,511674.5
고양시덕양구,3287291.4
고양시일산동구,2722980.0
고양시일산서구,922486.9
과천시,315666.0


In [197]:
filename = 'data/경기도_geo_simple.json'
geo_data = json.load(open(filename, encoding='utf-8'))

In [198]:
map = folium.Map([37.6, 127.1], zoom_start=8, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=df1.면적,                   
    columns=[df1.index, df1.면적],    
    fill_color='YlOrRd',                     
    key_on='properties.name'              
).add_to(map)
map

In [199]:
pop1 = pd.read_csv('../02.Pandas/data/주민등록인구집계현황.csv', encoding='euc-kr')
pop1 = pop1[['행정구역구분명','행정구역명','총 인구수']]
pop1.head()

,행정구역구분명,행정구역명,총 인구수
0,도,경기도,13589432
1,시군,경기도 가평군,62197
2,읍면동,경기도 가평군 가평읍,19555
3,읍면동,경기도 가평군 북면,3815
4,읍면동,경기도 가평군 상면,5669


In [200]:
pop1 = pop1[pop1.행정구역구분명.str.contains('시군') | pop1.행정구역구분명.str.contains('구')]
pop1.head()

,행정구역구분명,행정구역명,총 인구수
1,시군,경기도 가평군,62197
8,시군,경기도 고양시,1075202
9,구,경기도 고양시 덕양구,487874
31,구,경기도 고양시 일산동구,296590
44,구,경기도 고양시 일산서구,290738


In [201]:
gu_city = ['수원시', '고양시', '용인시', '성남시', '안산시', '안양시']
pop1.행정구역명 = pop1.행정구역명.str.strip()
drop_index = []
for index in pop1[pop1.행정구역구분명 == '시군'].index:
    if pop1.행정구역명[index].split()[-1] in gu_city:
        drop_index.append(index)
pop1.drop(drop_index, inplace=True)
pop1.head()

,행정구역구분명,행정구역명,총 인구수
1,시군,경기도 가평군,62197
9,구,경기도 고양시 덕양구,487874
31,구,경기도 고양시 일산동구,296590
44,구,경기도 고양시 일산서구,290738
56,시군,경기도 과천시,77775


In [202]:
pop1['시군구'] = pop1.행정구역명.apply(lambda x: ''.join(x.split()[1:]))
pop1.head()

,행정구역구분명,행정구역명,총 인구수,시군구
1,시군,경기도 가평군,62197,가평군
9,구,경기도 고양시 덕양구,487874,고양시덕양구
31,구,경기도 고양시 일산동구,296590,고양시일산동구
44,구,경기도 고양시 일산서구,290738,고양시일산서구
56,시군,경기도 과천시,77775,과천시


In [203]:
pop1 = pop1['시군구','총 인구수']
pop1.head()

KeyError: ('시군구', '총 인구수')